In [ ]:
import numpy as np
import matplotlib.pyplot as plt



## Exercice 5 : Vers la regression linéaire multiple et optimisation

On considère que l'on connait les notes sur une année de n élèves dans p matières, ainsi que leurs notes à un concours en fin d'annee. L'année suivante, on  se demande si on ne pourrait pas prédire la note des étudiants au concours en fonction de leurs notes annuelle afin d'estimer leurs chances de réussite au concours.


On va resoudre le problème à l'aide de la régression linéaire en dimension p>1 sans utiliser scikit-learn. 



### <span style="color:blue">QUESTION 5.1 :</span> 

A l'aide de la fonction 'SimulateObservations', simulez un jeu de donnees d'apprentissage [X_l,y_l] avec 30 observations et un jeu de test [X_t,y_t] avec 10 observations. Les observations seront en dimension p=10



In [ ]:
def SimulateObservations(n_train,n_test,p):
    """
    n_train: number of training obserations to simulate
    n_test: number of test obserations to simulate
    p: dimension of the observations to simulate
    """

    ObsX_train=20.*np.random.rand(n_train,p)
    ObsX_tst=20.*np.random.rand(n_test,p)

    RefTheta=np.random.rand(p)**3
    RefTheta=RefTheta/RefTheta.sum()
    print("The thetas with which the values were simulated is: "+str(RefTheta))

    ObsY_train=np.dot(ObsX_train,RefTheta.reshape(p,1))+1.5*np.random.randn(n_train,1)
    ObsY_tst=np.dot(ObsX_tst,RefTheta.reshape(p,1))+1.5*np.random.randn(n_test,1)

    return [ObsX_train,ObsY_train,ObsX_tst,ObsY_tst,RefTheta]


### <span style="color:blue">REPONSE 5.1 :</span> 

In [ ]:
...


### <span style="color:blue">QUESTION 5.2 :</span> 

On considere un modele linéaire en dimension p>1 pour mettre en lien les x[i,:] et les y[i], c'est a dire que np.dot(x[i,:],theta_optimal) doit etre le plus proche possible de y[i] sur l'ensemble des observations i. Dans le modèle linéaire multiple, theta_optimal est un vecteur de taille [p,1] qui pondère les différentes variables observées (ici les moyennes dans une matière). Coder alors une fonction qui calcule la moyenne des différences au carré entre ces valeurs en fonction de theta, *i.e.* la mean squared error (MSE) du modèle.

### <span style="color:blue">REPONSE 5.2 :</span> 

In [ ]:
def CptMSE(theta_test, X, y_true):
    y_pred = np.dot(X, theta_test)[:, np.newaxis]
    MSE = np.mean((y_true - y_pred)**2)
    return MSE

theta_test=np.random.rand(p)
theta_test=theta_test/theta_test.sum()

MSE_test=CptMSE(theta_test, ObsX_train, ObsY_train)


### <span style="color:blue">QUESTION 5.3 :</span> 

On va maintenant chercher le theta_test qui minimise cette fonction (il correspondra à theta_optimal), et ainsi résoudre le probleme d'apprentissage de regression lineaire multiple. Utiliser pour cela la fonction minimize de scipy.optimize


De manière importante, la recherche des paramètres *theta_optimal* sera effectuée en utilisant les observations d'apprentissage (*ObsX_train* et *ObsY_train* en sortie *SimulateObservations*). La MSE obtenue sur les observations d'apprentissage avec *theta_optimal* sera comparée à celle obtenue avec les observations de test (*ObsX_tst* et *ObsY_tst* en sortie *SimulateObservations*) avec le même *theta_optimal*. Que constatez vous ?



### <span style="color:blue">REPONSE 5.3 :</span> 

In [ ]:
from scipy.optimize import minimize

...

In [ ]:
...

On peut aussi optimiser le modèle par une descente de gradient.

In [ ]:
def CptMSE(X,y_true,theta_test):
    y_pred=np.dot(X,theta_test)[:, np.newaxis]
    #print(y_pred.shape)
    #print(y_true.shape)
    MSE=np.mean(np.power(y_pred-y_true,2.))

    return MSE


def gradientApprox(fct_to_minimize,theta_loc,X_loc,Y_loc,epsilon=1e-5):

    fx=fct_to_minimize(X_loc,Y_loc,theta_loc)
    #print(fx)
    ApproxGrad=np.zeros(np.size(theta_loc))
    veps=np.zeros(np.size(theta_loc))

    for i in range(np.size(theta_loc)):
        veps[:]=0.
        veps[i]+=epsilon
        ApproxGrad[i]=(fct_to_minimize(X_loc,Y_loc,theta_loc+veps)-fx)/epsilon
    return ApproxGrad


def analyticGradient(_, theta,x,y,epsilon=1e-5):
    shape = theta.shape
    theta = theta.reshape(-1, 1)
    grad_theta = (-2*np.dot((y-np.dot(x, theta)).T,x))/len(y)
    grad_theta = grad_theta.reshape(shape)
    return grad_theta

def GradientDescent(fct_to_minimize,theta_init,X_loc,Y_loc,alpha=0.01,N=100, approx=True):
    """
    Remark: the multiplicatory coefficient of the gradients will be "alpha" divided by the norm of the first gradient 
    """

    #init
    l_thetas=[theta_init]
    theta_curr=theta_init.copy()

    #run the gradient descent
    n=0
    while n<N:
        #approximate the gradient of fct_to_minimize w.r.t. theta_curr
        if approx:
            g=gradientApprox(fct_to_minimize,theta_curr,X_loc,Y_loc)
        else:
            g=analyticGradient(fct_to_minimize,theta_curr,X_loc,Y_loc)

        #set the multiplicatory coefficient of the gradients
        if n==0:
            NormFirstGrads=np.linalg.norm(g)
            coefMult=alpha/NormFirstGrads

        #update theta
        theta_curr=theta_curr-coefMult*g

        #save the current state and increment n
        l_thetas.append(theta_curr)
        n+=1

    return l_thetas

theta_init = np.abs(np.random.randn(p))/10.
thetas = GradientDescent(CptMSE, theta_init, ObsX_train, ObsY_train, approx=True)
theta_optim = thetas[-1]

mse_train = CptMSE(ObsX_train, ObsY_train, theta_optim)
mse_test = CptMSE(ObsX_test, ObsY_test, theta_optim)

mse_train_ref = CptMSE(ObsX_train, ObsY_train, RefTheta)
mse_test_ref = CptMSE(ObsX_test, ObsY_test, RefTheta)

print("Theta optimal --- ")
print("Base d'apprentissage - MSE : ", mse_train)
print("Base de test - MSE : ", mse_test)
print("")
print("Theta de référence --- ")
print("Base d'apprentissage - MSE : ", mse_train_ref)
print("Base de test - MSE : ", mse_test_ref)